# Implications of PF-Selection on Risk Measure

© Thomas Robert Holy 2019
<br>
Version 0.1.0

## Package Import

In [ ]:
#----------------------------------------------------------------------------------------
# Packages

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches
import operator 
import datetime as dt
import sys 
from scipy import stats
import math
import random

import riskmeasure_module as rm

## Settings

In [ ]:
#----------------------------------------------------------------------------------------
# Settings

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

SCREEN_WIDTH = 140 
centered = operator.methodcaller('center', SCREEN_WIDTH) 
pd.set_option('display.width', 125) 
plt.rcParams["figure.figsize"] = 15,12.5 

## Data read and DataFrame

In [ ]:
#----------------------------------------------------------------------------------------
# Data

datensatz1 = 'BAS.DE'
datensatz2 = ''
datensatz3 = 'example3'
datensatz4 = 'example4'
datensatz5 = 'example5'

dateinamen = [datensatz1,datensatz2,datensatz3,datensatz4,datensatz5]

#----------------------------------------------------------------------------------------
# Dataframe creation

kurse = []
for eintrag in dateinamen:
    kurs = pd.read_csv(str(eintrag) + '.csv',
                decimal='.',
                usecols=['Date','Adj Close'])
    kurse.append(kurs) 

for eintrag in kurse:
    eintrag['Date'] = pd.to_datetime(eintrag['Date']) 
    eintrag['Adj Close'] = pd.to_numeric(eintrag['Adj Close'], errors='coerce') 
    
#-------------------------------------------------
# Sharpe - PF

kurschart_0 = pd.DataFrame()

zaehler = 0
for eintrag in kurse:
    x = dateinamen[zaehler]
    kurschart_0['Rendite ' + str(x)] = eintrag['Adj Close']
    zaehler += 1

kurschart_0 = kurschart_0[:252]

#-------------------------------------------------
# Naive Diversifikation 

kurschart_1 = kurschart_0.pct_change()
kurschart_1['PF-Rendite'] = (kurschart_1.sum(axis = 1, skipna = True) / len(dateinamen))

## Monte-Carlo-Simulation: Sharpe PF, efficient Frontier

In [ ]:
#Source of efficient frontier: https://medium.com/python-data/effient-frontier-in-python-34b0c3043314, https://medium.com/python-data/efficient-frontier-portfolio-optimization-with-python-part-2-2-2fe23413ad94
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

trading_days = 253

# calculate daily and annual returns of the stocks
returns_daily = kurschart_0.pct_change()
returns_annual = returns_daily.mean() * trading_days

# get daily and covariance of returns of the stock
cov_daily = returns_daily.cov()
cov_annual = cov_daily * trading_days

port_returns = []
port_volatility = []
sharpe_ratio = []
stock_weights = []

# set the number of combinations for imaginary portfolios
num_assets = len(dateinamen)
num_portfolios = 10000

#----------------------------------------------------------------------------------------

# populate the empty lists with each portfolios returns,risk and weights

for single_portfolio in range(num_portfolios):
    weights = np.random.random(num_assets)
    weights /= np.sum(weights)
    returns = np.dot(weights, returns_annual)
    volatility = np.sqrt(np.dot(weights.T, np.dot(cov_annual, weights)))
    sharpe = returns / volatility
    sharpe_ratio.append(sharpe)
    port_returns.append(returns)
    port_volatility.append(volatility)
    stock_weights.append(weights)

#----------------------------------------------------------------------------------------

# a dictionary for Returns and Risk values of each portfolio
portfolio = {'Returns': port_returns,
             'Volatility': port_volatility,
             'Sharpe Ratio': sharpe_ratio}

#----------------------------------------------------------------------------------------

# extend original dictionary to accomodate each ticker and weight in the portfolio
for counter, symbol in enumerate(dateinamen):
    portfolio[symbol + ' Weight'] = [Weight[counter] for Weight in stock_weights]

# make a nice dataframe of the extended dictionary
df = pd.DataFrame(portfolio)

# get better labels for desired arrangement of columns
column_order = ['Returns', 'Volatility', 'Sharpe Ratio'] + [stock + ' Weight' for stock in dateinamen]

# reorder dataframe columns
df = df[column_order]

#----------------------------------------------------------------------------------------

# find min Volatility & max sharpe values in the dataframe (df)
min_volatility = df['Volatility'].min()
max_sharpe = df['Sharpe Ratio'].max()

# use the min, max values to locate and create the two special portfolios
sharpe_portfolio = df.loc[df['Sharpe Ratio'] == max_sharpe]
min_variance_port = df.loc[df['Volatility'] == min_volatility]

# plot frontier, max sharpe & min Volatility values with a scatterplot
plt.style.use('seaborn-dark')
df.plot.scatter(x='Volatility', y='Returns', c='Sharpe Ratio',
                cmap='RdYlGn', edgecolors='black', figsize=(10, 8), grid=True)
plt.scatter(x=sharpe_portfolio['Volatility'], y=sharpe_portfolio['Returns'], c='red', marker='D', s=200)
plt.scatter(x=min_variance_port['Volatility'], y=min_variance_port['Returns'], c='blue', marker='D', s=200 )
plt.xlabel('Volatility (Std. Deviation)')
plt.ylabel('Expected Returns')
plt.title('Efficient Frontier')
plt.show()

#----------------------------------------------------------------------------------------

# print the details of the 2 special portfolios
print('#--------------------------------------------------------------------------------------------------------------------------------------------#')
print('Minimum-Varianz-Portfolio: \n' + str(min_variance_port.T))
print('#--------------------------------------------------------------------------------------------------------------------------------------------#')
print('Optimales Sharpe-Portfolio: \n' + str(sharpe_portfolio.T))
print('#--------------------------------------------------------------------------------------------------------------------------------------------#')

## Clean Data

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Vorbereitung

sharpe_portfolio = sharpe_portfolio.values.tolist()
sharpe_portfolio = sharpe_portfolio[0][3::]

kurschart_0 = kurschart_0.pct_change()
kurschart_0 = kurschart_0.multiply(sharpe_portfolio, axis = 1)
kurschart_0['PF-Rendite'] = kurschart_0.sum(axis = 1, skipna = True)

#--------------------------------

sharpe_values_PF = kurschart_0['PF-Rendite'].values.tolist()
sharpe_values_PF = np.array(sharpe_values_PF)
sharpe_values_PF = sharpe_values_PF[np.logical_not(np.isnan(sharpe_values_PF))]
sharpe_values_PF = sharpe_values_PF[sharpe_values_PF != 0.0]

#--------------------------------

naiv_values_PF = kurschart_1['PF-Rendite'].values.tolist()
naiv_values_PF = np.array(naiv_values_PF)
naiv_values_PF = naiv_values_PF[np.logical_not(np.isnan(naiv_values_PF))]
naiv_values_PF = naiv_values_PF[naiv_values_PF != 0.0]

## Optimal Sharpe-PF and Risk Measure

In [ ]:
#----------------------------------------------------------------------------------------
# Optimales PF nach Sharpe

print('##############################################################################################################################################')
print('|' + centered('Optimales PF nach Sharpe') + '| ')
print('##############################################################################################################################################')

#----------------------------------------------------------------------------------------
mu_sharpe_PF = np.mean(sharpe_values_PF)
std_sharpe_PF = np.std(sharpe_values_PF)

print('#--------------------------------------------------------------------------------------------------------------------------------------------#')
print('|' + centered('[INFO] Die Porfolio-Rendite hat einen Erwartunswert i.H.v. ' + str(mu_sharpe_PF) + '.') + '| ')
print('#--------------------------------------------------------------------------------------------------------------------------------------------#')
print('|' + centered('[INFO] Das Porfolio hat eine Standardabweichung i.H.v. ' + str(std_sharpe_PF) + '.') + '| ')
print('#--------------------------------------------------------------------------------------------------------------------------------------------#')

mini_values_PF = min(min(naiv_values_PF), min(sharpe_values_PF))
maxi_values_PF = max(max(naiv_values_PF), max(sharpe_values_PF)) 

bins = len(naiv_values_PF)

#----------------------------------------------------------
# Historische Simulation

def hist_sim(values, bins):
    H, X1 = np.histogram(values, bins, density=True)
    dx = X1[1] - X1[0]
    F1 = np.cumsum(H) * dx 
    plt.plot(X1[1:], F1) 
hist_sim(sharpe_values_PF, bins)

#----------------------------------------------------------
# Varianz-Kovarianz-Methode

def var_co_var_sim(mini_values_PF, maxi_values_PF, bins, mu, std):
    global var_covar_results
    
    array = np.array(np.arange(0.0001, 1, 0.0001))
    var_covar_results = stats.norm.ppf(array, mu, std)

    var_covar_range = np.linspace(mini_values_PF, maxi_values_PF, bins)
    plt.plot(var_covar_range, stats.norm.cdf(var_covar_range, mu, std))
var_co_var_sim(mini_values_PF, maxi_values_PF, bins, mu_sharpe_PF, std_sharpe_PF)

#----------------------------------------------------------
# Restliche Einstellungen für die Grafik

def easy_plot():
    plt.xlabel('Rendite') 
    plt.ylabel('Wahrscheinlichkeit') 
    blue_patch = mpatches.Patch(color='blue', label='Historische Simulation') 
    orange_patch = mpatches.Patch(color='orange', label='Varianz-Kovarianzmethode') 
    plt.legend(handles=[orange_patch, blue_patch]) 
    plt.title('Verteilungsfunktion: Historische Simulation versus Varianz-Kovarianz-Methode')
    plt.grid() 
    plt.axhline(0, color='black') 
    plt.axvline(0, color='black') 
    plt.show() 
easy_plot() 

#---------------------------------------------------------------------------------------
# Risk Measure - Historische Simulation

print('#--------------------------------------------------------------------------------------------------------------------------------------------#')
print('Sharpe: Risk Measure - Historische Simulation')
print('#--------------------------------------------------------------------------------------------------------------------------------------------#')

def risk(values, alpha, gamma):
    # VaR:
    VaR = rm.risk_measure()
    VaR.VaR(values, alpha)
    x = VaR.VaR
    print('VaR = ' + str(x) + '\n')

    # CVaR:
    CVaR = rm.risk_measure()
    CVaR.CVaR(values, alpha)
    x = CVaR.CVaR
    print('CVaR = ' + str(x) + '\n')

    # Power:
    psrm = rm.risk_measure()
    psrm.Power(values, gamma)
    x, y = psrm.power
    print('Power-Spektrales Risikomaß:')
    print('EW = ' + str(y))
    print('Risk = ' + str(x))
    print('#--------------------------------------------------------------------------------------------------------------------------------------------#')

alpha = 0.2
gamma = 0.5

risk(sharpe_values_PF, alpha, gamma)

#---------------------------------------------------------------------------------------
# Risk Measure - Varianz-Kovarianz-Methode

print('Sharpe: Risk Measure - Varianz-Kovarianz-Methode')
print('#--------------------------------------------------------------------------------------------------------------------------------------------#')

risk(var_covar_results, alpha, gamma)

## Naive diversification and Risk Measure

In [ ]:
#--------------------------------------------------------------------------
# Naive Diversifikation

print('##############################################################################################################################################')
print('|' + centered('Naive Diversifikation') + '| ')
print('##############################################################################################################################################')

#----------------------------------------------------------------------------------------
mu_naiv_PF = np.mean(naiv_values_PF)
std_naiv_PF = np.std(naiv_values_PF)

print('#--------------------------------------------------------------------------------------------------------------------------------------------#')
print('|' + centered('[INFO] Die Porfolio-Rendite hat einen Erwartunswert i.H.v. ' + str(mu_naiv_PF) + '.') + '| ')
print('#--------------------------------------------------------------------------------------------------------------------------------------------#')
print('|' + centered('[INFO] Das Porfolio hat somit eine Standardabweichung i.H.v. ' + str(std_naiv_PF) + '.') + '| ')
print('#--------------------------------------------------------------------------------------------------------------------------------------------#')

#----------------------------------------------------------
# Historische Simulation

hist_sim(naiv_values_PF, bins)

#----------------------------------------------------------
# Varianz-Kovarianz-Methode

var_co_var_sim(mini_values_PF, maxi_values_PF, bins, mu_naiv_PF, std_naiv_PF)

#----------------------------------------------------------
# Restliche Einstellungen für die Grafik

easy_plot() 

#---------------------------------------------------------------------------------------
# Risk Measure - Historische Simulation

print('#--------------------------------------------------------------------------------------------------------------------------------------------#')
print('Naiv: Risk Measure - Historische Simulation')
print('#--------------------------------------------------------------------------------------------------------------------------------------------#')

risk(naiv_values_PF, alpha, gamma)

#---------------------------------------------------------------------------------------
# Risk Measure - Varianz-Kovarianz-Methode

print('Naiv: Risk Measure - Varianz-Kovarianz-Methode')
print('#--------------------------------------------------------------------------------------------------------------------------------------------#')

risk(var_covar_results, alpha, gamma)